In [1]:
# # coding: utf-8

# Suite à une perte de données, ce fichier n'est pas celui qui a servi à générer les données utilisées dans l'appli.
# Il s'agit d'une version préliminaire qui a ensuite été améliorée mais dont je n'ai plus le fichier. 
# Toutes mes excuses pour l'inconvénient, je fais tout de même le choix de mettre celui ci en ligne en 
# attendant de le remodifier plus tard lorsque j'en aurai le temps.

# Modifications perdues et à refaire par la suite : 
# remodifier lte lge dans requete API, optimiser temps de generation du script
# déclarations multiples
# utilisation de compréhensions de listes / map() pour accélerer le script



import json
import requests
import pandas as pd
from calendar import monthrange
pd.options.mode.chained_assignment = None  # default='warn'

# choix du mois que l'on veut récupérer par API 
annee = int(input("pour quelle année ? (format YYYY)?"))
mois_recherche = int(input("pour quel mois (de 1 à 12) ?")) # sert ensuite pour générer le bon lien à requeter.

# récupération du nombre de jours par mois au format dict (selon la variable "annee")
mois = []
nbjour = []
for i in range(1,13) :
    mois.append(i)
    nbjour.append(monthrange(annee, i)[1])
mois_nb_jours = {mois[i]: nbjour[i] for i in range(len(mois))}

# GENERATION DE MES LIENS DE REQUETE + PREP DONNEES AVANT CREATION historique_velov 

## parties fixes de mes liens de requête API pour toute la boucle for qui suit

link_endpoint = "https://download.data.grandlyon.com/ws/timeseries/jcd_jcdecaux.historiquevelov/all.json?horodate__gte="
max_features = "&maxfeatures=200000"


for i in range(1,mois_nb_jours[mois_recherche]+1) :
    date_min = str(annee) + "-" + str(mois_recherche) + "-" + str(i)
    
    if (mois_recherche == 12 and i == 31) : 
        date_max = "&horodate__lt=" + str(annee+1) + "-01-01"
        lien_entier = link_endpoint + date_min + date_max + max_features

    elif i == mois_nb_jours[mois_recherche] : 
        date_max = "&horodate__lt=" + str(annee) + "-" + str(mois_recherche+1) + "-01"
        lien_entier = link_endpoint + date_min + date_max + max_features

    else : 
        date_max = "&horodate__lt=" + str(annee) + "-" + str(mois_recherche) + "-" + str(i+1) 
        lien_entier = link_endpoint + date_min + date_max + max_features

        

for i in range(1,mois_nb_jours[mois_recherche]+1) :
    date_min = str(annee) + "-" + str(mois_recherche) + "-" + str(i)
    # pour chaque jour je récupère un lien différent 
    if (mois_recherche == 12 and i == 31) : 
        date_max = "&horodate__lt=" + str(annee+1) + "-01-01"
        lien_entier = link_endpoint + date_min + date_max + max_features
    elif i == mois_nb_jours[mois_recherche] : 
        date_max = "&horodate__lt=" + str(annee) + "-" + str(mois_recherche+1) + "-01"
        lien_entier = link_endpoint + date_min + date_max + max_features
    else : 
        date_max = "&horodate__lt=" + str(annee) + "-" + str(mois_recherche) + "-" + str(i+1) 
        lien_entier = link_endpoint + date_min + date_max + max_features
        
# je reste dans ma boucle : pour chaque jour je requete le lien obtenu :       
    r = requests.get(lien_entier)
    
# mise en place d'un try except pour gérer les erreurs dûes au format de certaines lignes du JSON récupéré par l'API
    try : 
        data_endpoint = json.loads(r.text)
        historique_velov_jour = pd.json_normalize(data_endpoint, record_path="values")
        
        # suppression lignes inutiles (pour ce que je veux faire ici) 
        historique_velov_jour.drop(historique_velov_jour[historique_velov_jour["status"]=="CLOSED"].index,inplace=True)
        
        # suppression colonnes inutiles
        historique_velov_jour = historique_velov_jour.drop(columns=['overflow_stands', "status", 
                   'main_stands.availabilities.bikes',
                   'main_stands.availabilities.electricalBikes',
                   'main_stands.availabilities.electricalInternalBatteryBikes',
                   'main_stands.availabilities.electricalRemovableBatteryBikes',
                   'main_stands.availabilities.mechanicalBikes',
                   'main_stands.availabilities.stands', 'main_stands.capacity',
                   'total_stands.availabilities.electricalInternalBatteryBikes',
                   'total_stands.availabilities.electricalRemovableBatteryBikes',
                   'total_stands.availabilities.mechanicalBikes',
                   'overflow_stands.availabilities.electricalBikes',
                   'overflow_stands.availabilities.electricalInternalBatteryBikes',
                   'overflow_stands.availabilities.electricalRemovableBatteryBikes',
                   'overflow_stands.availabilities.mechanicalBikes',
                   'overflow_stands.availabilities.bikes',
                   'overflow_stands.availabilities.stands',
                   'overflow_stands.capacity',])
        
        #renommage colonnes
        historique_velov_jour.rename(columns={'total_stands.availabilities.bikes' : 'availabilities.all.types',
                   'total_stands.availabilities.electricalBikes' : 'availabilities.electricalBikes',
                   'total_stands.availabilities.stands' : "availabilities.stands", 
                   'total_stands.capacity' :"capacity"}, inplace=True)
        
        # je récupère une liste des numéros de station qui ont été récupérées sur le mois entier
        liste_numero_station = list(historique_velov_jour["number"].unique())
        
        # passage des dates au format datetime
        historique_velov_jour['horodate'] = pd.to_datetime(historique_velov_jour['horodate'], format='%Y-%m-%d %H:%M:%S%Z')
        historique_velov_jour.sort_values(by=["number", "horodate"], inplace=True)
    
        
        # je récupère un df pour chaque station dans ma liste de stations ouvertes pour le mois que j'ai choisi précédement : 
        for station in liste_numero_station :
            mask = historique_velov_jour["number"] == station
            chaque_station = historique_velov_jour.loc[mask]
            chaque_station['horodate'] = pd.to_datetime(chaque_station['horodate'], format='%Y-%m-%d %H:%M:%S%Z')
            # afin de visualiser de quelle maniène la gestion des frequences est faite sur la ligne de code utilisant ".asfreq", je duplique ma colonne horodate
            chaque_station.insert(1, "heure_exacte", chaque_station['horodate'])
            chaque_station.sort_values(by="horodate", inplace=True)
            
            # on mets les datetimes en index pour ensuite pouvoir utiliser .asfreq
            chaque_station.set_index('horodate',inplace=True)
            # à savoir : les horaires de mon index sont modifiés suite à .asfreq 
            chaque_station=chaque_station.asfreq('15T', method='bfill') 

            # pour la premiere station traitée je crée un nouveau df, pour les stations suivante je concatène avec le df crée en 0
            if station == liste_numero_station[0] : 
                station_recuperee = chaque_station
            else : 
                station_recuperee = pd.concat([station_recuperee,chaque_station])
                
        # pour le premier jour du mois je crée un df, pour les autres je concatene
        # je choisi de récupérer deux df différents afin de comparer la cohérence de mes résultats, la différence de nombre de lignes 
        if i == 1 :
            historique_velov_mois = historique_velov_jour # ici df qui contient la totalité de la requête API
            historique_mois_15min = station_recuperee # ici df retravaillé qui contient uniquement les lignes par tranches de 15 minutes afin d'alleger le df
        else : 
            historique_mois_15min = pd.concat([historique_mois_15min,station_recuperee])
            historique_velov_mois = pd.concat([historique_velov_mois,historique_velov_jour])
            
    except KeyError as e : 
        print ("KeyError pour date du : " + str(i) + "/" + str(mois_recherche) + "/" + str(annee))
        print(e)
    except json.decoder.JSONDecodeError as e :    
        print ("JSONDecodeError pour date du : " + str(i) + "/" + str(mois_recherche) + "/" + str(annee))
        
historique_mois_15min.reset_index(inplace=True)       
# Export en csv

if mois_recherche <10 : 
    historique_mois_15min.to_csv(str(annee) + "-0" + str(mois_recherche) +"-allege15min.csv", index=False)
else :
    historique_mois_15min.to_csv(str(annee) + "-" + str(mois_recherche) +"-allege15min.csv", index=False)


JSONDecodeError pour date du : 14/2/2023
KeyError pour date du : 18/2/2023
'status'


In [9]:
historique_mois_15min

,horodate,heure_exacte,number,availabilities.all.types,availabilities.electricalBikes,availabilities.stands,capacity
0,2023-02-01 00:04:30+01:00,2023-02-01 00:04:30+01:00,555,0,0,3,5
1,2023-02-01 00:19:30+01:00,2023-02-01 00:24:41+01:00,555,0,0,3,5
2,2023-02-01 00:34:30+01:00,2023-02-01 00:34:46+01:00,555,0,0,3,5
3,2023-02-01 00:49:30+01:00,2023-02-01 00:54:57+01:00,555,0,0,3,5
4,2023-02-01 01:04:30+01:00,2023-02-01 01:05:02+01:00,555,0,0,3,5
...,...,...,...,...,...,...,...
973005,2023-02-28 22:50:27+01:00,2023-02-28 22:50:28+01:00,34002,7,4,8,15
973006,2023-02-28 23:05:27+01:00,2023-02-28 23:11:26+01:00,34002,7,4,8,15
973007,2023-02-28 23:20:27+01:00,2023-02-28 23:21:31+01:00,34002,7,4,8,15
973008,2023-02-28 23:35:27+01:00,2023-02-28 23:41:42+01:00,34002,7,4,8,15
